In [1]:
import os
import matplotlib.pyplot

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds


In [2]:
(ds_train, ds_test), ds_info = tfds.load(
    "mnist",
    split=["train", "test"],
    shuffle_files=True,
    as_supervised=True,  # will return tuple (img, label) otherwise dict
    with_info=True,  # able to get info about dataset
)


Dl Completed...:   0%|          | 0/5 [00:00<?, ? file/s]

Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


In [3]:
def normalize_img(image, label):
    """Normalizes images"""
    return tf.cast(image, tf.float32) / 255.0, label


In [4]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 128

# Setup for train dataset
ds_train = ds_train.map(normalize_img, num_parallel_calls=AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits["train"].num_examples)
ds_train = ds_train.batch(BATCH_SIZE)
ds_train = ds_train.prefetch(AUTOTUNE)


In [5]:
model = keras.Sequential(
    [
        keras.Input((28, 28, 1)),
        layers.Conv2D(32, 3, activation="relu"),
        layers.Flatten(),
        tf.keras.layers.Dense(10, activation="softmax"),
    ]
)


In [29]:
filepath ="/content/checkpoint{epoch:02d}.h5"

In [30]:
save_callback = keras.callbacks.ModelCheckpoint(
    filepath, save_weights_only=True, monitor="train_acc", save_best_only=False,  save_freq="epoch"
)


In [23]:
lr_scheduler = keras.callbacks.ReduceLROnPlateau(
    monitor="loss", factor=0.1, patience=3, mode="max", verbose=1
)


In [24]:
# class OurOwnCallback(keras.callbacks.Callback):
#     def on_epoch_end(self, epoch, logs=None):
#         if logs.get("accuracy") > 1:
#             print("Accuracy over 70%, quitting training")
#             self.model.stop_training = True


In [25]:
model.compile(
    optimizer=keras.optimizers.Adam(0.01),
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"],
)


In [31]:
model.fit(
    ds_train,
    epochs=4,
    callbacks=[save_callback],
    verbose=2,
)


Epoch 1/4
469/469 - 2s - loss: 0.0068 - accuracy: 0.9988 - 2s/epoch - 3ms/step
Epoch 2/4
469/469 - 2s - loss: 0.0052 - accuracy: 0.9988 - 2s/epoch - 4ms/step
Epoch 3/4
469/469 - 1s - loss: 0.0044 - accuracy: 0.9991 - 1s/epoch - 3ms/step
Epoch 4/4
469/469 - 1s - loss: 0.0073 - accuracy: 0.9987 - 1s/epoch - 3ms/step
